<a href="https://colab.research.google.com/github/yutakasawai/Pytorch_sandbox/blob/master/PytorchCatalyst_ResNet_CIFAR10_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list | grep cata

catalogue                1.0.0          


In [2]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version $VERSION > /dev/null
!pip install git+https://github.com/catalyst-team/catalyst@master > /dev/null
#!pip install -U catalyst

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  27500      0 --:--:-- --:--:-- --:--:-- 27648
Copying gs://tpu-pytorch/wheels/torch-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 1 objects/79.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][106.6 MiB/106.6 MiB]                                                
Operation completed over 1 objects/106.6 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-1.5-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.5 MiB/  2.5 MiB]                                                
Operation completed over 1 objects/2.5 MiB.                                      
ERROR: fastai 1.0.61 requires torchvision, wh

In [4]:
import os
import random

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from catalyst import dl, utils
from catalyst.utils import metrics
from catalyst.dl.callbacks import AccuracyCallback
from catalyst.core.callbacks.early_stop import EarlyStoppingCallback

In [5]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

    
seed_everything(1192)

In [6]:
device = utils.get_device()  # <--------- TPU device
device

device(type='xla', index=1)

In [7]:
val_transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.RandomCrop(224,padding=4),
        transforms.RandomHorizontalFlip(),

        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_dataset = datasets.CIFAR10(root="./data",train=True,download=True,transform=train_transform)
val_dataset = datasets.CIFAR10(root="./data",train=False,download=True,transform=val_transform)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# val_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,),(0.5,))
# ])

# train_transform = transforms.Compose([
#         # transforms.RandomHorizontalFlip(),
#         # transforms.ColorJitter(),
#         # transforms.RandomRotation(10),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,),(0.5,))
# ])

# train_dataset = datasets.CIFAR10(root="./data",train=True,download=True,transform=train_transform)
# val_dataset = datasets.CIFAR10(root="./data",train=False,download=True,transform=val_transform)

In [8]:
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        trained_model = models.resnet18(pretrained=True)
        self.trained_model = nn.Sequential(*list(trained_model.children())[:-1])
        in_features = trained_model.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)
    def forward(self,x):
        x = self.trained_model(x)
        x = self.drop(x.view(-1,self.final.in_features))
        x = self.final(x)
        return x

In [9]:
model = ResNetModel().to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [10]:
print(model)

ResNetModel(
  (trained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [11]:
# for param in model.parameters():
#     param.requires_grad = False

In [12]:
for x,y in model.named_children():
    print(x)

trained_model
drop
final


In [13]:
print(model)

ResNetModel(
  (trained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [14]:
names = ("plane","car","bird","cat","deer","dog","frog","horse","ship","truck")

In [15]:
use_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

BATCH_SIZE = 128
train_dataloader = DataLoader(train_dataset,batch_size= BATCH_SIZE ,shuffle=True, **kwargs)
val_dataloader = DataLoader(val_dataset,batch_size= BATCH_SIZE ,shuffle=False, **kwargs)


In [16]:
criterion = nn.CrossEntropyLoss().to(device)
param_groups = [{'params':model.trained_model.parameters(),'lr':0.0001},
                {'params':model.final.parameters(),'lr':0.001}]
#optimizer = optim.Adam(model.fc.parameters(),lr=0.001)
optimizer = optim.Adam(param_groups)

In [17]:
import collections
loaders = collections.OrderedDict()
loaders["train"]=train_dataloader
loaders["valid"]=val_dataloader

In [20]:
from catalyst.dl.runner import SupervisedRunner
runner = SupervisedRunner(device=device)

In [21]:
# for i,l in train_dataloader:
#     print(i,l)
# i.size()

In [22]:
!rm -rf ./logs

In [23]:
runner.train(model=model,
            optimizer=optimizer,
             criterion=criterion,
            loaders=loaders,
            callbacks=[AccuracyCallback(num_classes=10,accuracy_args=[1])],
            main_metric="accuracy01",minimize_metric=False,
             logdir="./logs",
             num_epochs=5,
             verbose=True,
             
             
            )

1/5 * Epoch (train): 100% 391/391 [07:45<00:00,  1.19s/it, accuracy01=0.900, loss=0.255]
1/5 * Epoch (valid): 100% 79/79 [01:35<00:00,  1.21s/it, accuracy01=0.938, loss=0.125]
[2020-08-02 00:08:59,861] 
1/5 * Epoch 1 (_base): lr=0.0001 | momentum=0.9000
1/5 * Epoch 1 (train): accuracy01=0.8760 | loss=0.3667
1/5 * Epoch 1 (valid): accuracy01=0.9245 | loss=0.2120
2/5 * Epoch (train): 100% 391/391 [07:29<00:00,  1.15s/it, accuracy01=0.913, loss=0.214]
2/5 * Epoch (valid): 100% 79/79 [01:18<00:00,  1.01it/s, accuracy01=1.000, loss=0.046]
[2020-08-02 00:17:50,389] 
2/5 * Epoch 2 (_base): lr=0.0001 | momentum=0.9000
2/5 * Epoch 2 (train): accuracy01=0.9490 | loss=0.1507
2/5 * Epoch 2 (valid): accuracy01=0.9433 | loss=0.1711
3/5 * Epoch (train): 100% 391/391 [07:18<00:00,  1.12s/it, accuracy01=0.988, loss=0.055]
3/5 * Epoch (valid): 100% 79/79 [01:17<00:00,  1.01it/s, accuracy01=0.938, loss=0.091]
[2020-08-02 00:26:28,222] 
3/5 * Epoch 3 (_base): lr=0.0001 | momentum=0.9000
3/5 * Epoch 3 (tra